# BASE DE DATOS

In [2]:
import mysql.connector
import pandas as pd
import json

import random, string
from random import randrange
from datetime import datetime


import warnings
warnings.filterwarnings("ignore")

In [3]:
db = mysql.connector.connect(
    host = "localhost",
    user="root",
    password="1234",
    
)

In [4]:
def sql(_query):
    return pd.read_sql_query(_query, db)

In [5]:
cursor = db.cursor()

In [6]:
cursor.execute("CREATE DATABASE StarWars_JSON")
db.commit()

In [7]:
db = mysql.connector.connect(
    host = "localhost",
    user="root",
    password="1234",
    database= "StarWars_JSON"
)

In [8]:
cursor = db.cursor()

In [9]:
query= """
    CREATE TABLE Tipo_armas (    
        id_tipo_arma INT PRIMARY KEY NOT NULL AUTO_INCREMENT,
        tipo_arma VARCHAR(50));
"""
cursor.execute(query)
db.commit()

In [10]:
query= """
CREATE TABLE Comerciantes (
    id_comerciante INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
    comerciante VARCHAR (50)    
);
"""
cursor.execute(query)
db.commit()

In [11]:
query= """
CREATE TABLE Divisiones (
  id_division INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
  
  division VARCHAR (50),
  naves INTEGER,
  droides INTEGER,
  veh_terrestres INTEGER,
  soldados INTEGER,
  bajas INTEGER);
"""
cursor.execute(query)
db.commit()

In [12]:
query= """

CREATE TABLE lideres (
      id_lider INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
      
      lider VARCHAR (50)  
);
"""

cursor.execute(query)
db.commit()

In [13]:
query= """

CREATE TABLE Causas (
  id_causa INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
  causa VARCHAR (50)
);
"""
cursor.execute(query)
db.commit()

In [14]:
query= """

CREATE TABLE Territorios (
  id_territorio INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
  territorio VARCHAR (50)
);
"""
cursor.execute(query)
db.commit()

In [15]:
query= """

CREATE TABLE Religiones (
  id_religion INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
  religion VARCHAR (50)
);
"""
cursor.execute(query)
db.commit()

In [16]:
query= """

CREATE TABLE Suministros (
  id_suministro INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
  suministro VARCHAR (50)
);
"""
cursor.execute(query)
db.commit()

In [17]:
query= """

CREATE TABLE Razas (
  id_raza INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
  raza VARCHAR (50)
);
"""
cursor.execute(query)
db.commit()

In [18]:
query= """
CREATE TABLE Planetas (
  id_planeta INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
  planeta VARCHAR (50)
);
"""
cursor.execute(query)
db.commit()

In [20]:
query= """

CREATE TABLE Org_mediadoras (

      id_org INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
      mediador VARCHAR (50),
      tipo VARCHAR(50),
      organizacion VARCHAR(50),
      fecha_entrada DATE,
      fecha_salida DATE,
      
      id_lider_json_fk JSON NOT NULL,
      
     JSON_EXTRACT(Org_mediadoras.id_lider_json_fk, '$.Lider1')) REFERENCES Lideres(id_lider),
   
);
"""
cursor.execute(query)
db.commit()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Org_mediadoras.id_lider_json_fk, '$.Lider1')) REFERENCES Lideres(id_lider)),
   ' at line 12

In [ ]:
query= """

CREATE TABLE Bandos (
      id_bando INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
      
      bando VARCHAR (250),
      fecha_entrada DATE,
      fecha_salida DATE,
      
      id_division_json_fk JSON NOT NULL,
      id_comerciante_json_fk JSON NOT NULL,
      id_lider_json_fk JSON NOT NULL);
"""
cursor.execute(query)
db.commit()

In [ ]:
query= """

CREATE TABLE Causa_conflictos (
    id_causa_conflicto INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
    id_causa_fk INT,
    
    interes_json_fk JSON NOT NULL,
       
    FOREIGN KEY (id_causa_fk) REFERENCES Causas(id_causa));
"""
cursor.execute(query)
db.commit()

In [ ]:
query= """

CREATE TABLE Armas (

    id_arma INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
    arma VARCHAR (50),
    indicador_dest INT CHECK (indicador_dest BETWEEN 1 AND 10),
    
    id_tipo_arma_fk INT,
    id_comerciante_json_fk JSON NOT NULL,
    
    FOREIGN KEY (id_tipo_arma_fk) REFERENCES Tipo_armas(id_tipo_arma)
);
"""
cursor.execute(query)
db.commit()

In [ ]:
query= """
CREATE TABLE conflictos (

    id_conflicto INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
    conflicto VARCHAR(50),

    muertos INTEGER,
    heridos INTEGER,
    id_causa_conflicto_fk INT,
    id_planeta_fk INT,
    id_org_fk INT,

    id_bando_json_fk JSON NOT NULL,

    FOREIGN KEY (id_causa_conflicto_fk) REFERENCES causa_conflictos(id_causa_conflicto),
    FOREIGN KEY (id_planeta_fk) REFERENCES planetas(id_planeta),
    FOREIGN KEY (id_org_fk) REFERENCES Org_mediadoras(id_org));
"""
cursor.execute(query)
db.commit()

# CONSULTAS

## El grupo armado que primero se implicó en el conflicto con mayor número de muertos y heridos hasta la fecha se de hoy (9237) sumado al conflicto de causa religiosa en el que ha habido más intervenciones mediadoras de organizaciones mediadoras.

In [ ]:
sql("""
SELECT * FROM conflictos WHERE (heridos + muertos) = (SELECT MAX(heridos + muertos) FROM conflictos)
UNION
SELECT * FROM conflictos WHERE (id_causa_conflicto_fk=3 OR id_causa_conflicto_fk=4) AND (id_org_fk!=0)
""")

## Se ha producido un conflicto en Anaxes, con el resultado de 1200 muertos y 5000 heridos. Este acto está relacionado con el conflicto en el que participa el grupo armado "Jedi".

In [ ]:
query = """
INSERT INTO conflictos (conflicto, muertos, heridos,id_planeta_fk,id_bando_json_fk)

VALUES (%s, %s, %s, %s, %s);
"""
values= ('Batalla de Anaxes II',1200,5000,17,json.dumps({'Bando1':5}))

try:
    cursor.execute(query, values)
    db.commit()
except Exception as e:
            print(e)

## Mostrar el número total de víctimas (muertos y heridos) causadas por los conflictos bélicos en cada país, incluidos los países en los que no ha habido víctimas.

In [ ]:
sql("""
SELECT planetas.planeta,
COALESCE(SUM(conflictos.muertos + conflictos.heridos), 0) AS victimas 
FROM planetas
LEFT JOIN conflictos ON conflictos.id_planeta_fk = planetas.id_planeta
GROUP BY planetas.planeta
ORDER BY victimas DESC
""")

COALESCE==>se usa para devolver 0 en caso de que un planeta no haya tenido ningún conflicto y en ese caso no habría ninguna suma.

## Mostrar los nombres de los conflictos en los que se han producido intervenciones de mediación en 8010, 8045 y 8290.

In [ ]:
sql("""
SELECT Conflicto FROM conflictos
INNER JOIN org_mediadoras ON conflictos.id_org_fk = org_mediadoras.id_org
WHERE org_mediadoras.fecha_entrada >= '8010-01-01' AND org_mediadoras.fecha_entrada <= '8010-12-31' 
OR org_mediadoras.fecha_entrada >= '8045-01-01' AND org_mediadoras.fecha_entrada <= '8045-12-31' 
OR org_mediadoras.fecha_entrada >= '8290-01-01' AND org_mediadoras.fecha_entrada <= '8290-12-31'
""")

## Mostrar los nombres de los conflictos que mantienen el conflicto religioso activo más largo.

In [ ]:
sql("""
SELECT 

planetas.planeta,

conflictos.conflicto,

causas.causa,
bandos.bando,
bandos.fecha_entrada,
bandos.fecha_salida,

ROUND(DATEDIFF(bandos.fecha_salida, bandos.fecha_entrada)/365) AS tiempo_participacion

FROM Conflictos

INNER JOIN planetas ON
conflictos.id_planeta_fk = planetas.id_planeta

INNER JOIN causa_conflictos ON
conflictos.id_causa_conflicto_fk = causa_conflictos.id_causa_conflicto

INNER JOIN causas ON
causa_conflictos.id_causa_conflicto = causas.id_causa

LEFT JOIN bandos ON
bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando1')
 
WHERE causas.causa='Religiones'
ORDER BY tiempo_participacion DESC
LIMIT 1;
 
""")

## Mostrar los nombres de los grupos armados que se han retirado de todos los conflictos en los que han participado, junto con la fecha en que se retiraron del último conflicto.

In [ ]:
sql("""
SELECT

base.bando,

MAX(base.fecha_bando1) AS Ultima_Fecha

FROM(
    SELECT 
        conflictos.conflicto,
        bandos.bando,
        MAX(bandos.fecha_salida) as fecha_bando1
    FROM Conflictos
    LEFT JOIN bandos ON
     bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando1')
    WHERE fecha_salida > 0
    GROUP BY conflictos.conflicto,bandos.bando
    UNION

    SELECT 
        conflictos.conflicto,
        bandos.bando,
        MAX(bandos.fecha_salida) as fecha_bando2
    FROM Conflictos
    LEFT JOIN bandos ON
     bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando2')
    WHERE fecha_salida > 0
    GROUP BY conflictos.conflicto,bandos.bando
    UNION

    SELECT 
        conflictos.conflicto,
        bandos.bando,
        MAX(bandos.fecha_salida) as fecha_bando3
    FROM Conflictos
    LEFT JOIN bandos ON
     bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando3')
    WHERE fecha_salida > 0
    GROUP BY conflictos.conflicto, bandos.bando
    UNION

    SELECT 
        conflictos.conflicto,
        bandos.bando,
        MAX(bandos.fecha_salida) as fecha_bando4
    FROM Conflictos
    LEFT JOIN bandos ON
     bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando4')
    WHERE fecha_salida > 0
    GROUP BY conflictos.conflicto, bandos.bando) base    
    GROUP BY base.bando;
""")


Subconsulta

In [ ]:
sql("""   
    SELECT 
        conflictos.conflicto,
        bandos.bando,
        MAX(bandos.fecha_salida) as fecha_bando
    FROM Conflictos
    LEFT JOIN bandos ON
     bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando1')
    WHERE fecha_salida > 0
    GROUP BY conflictos.conflicto,bandos.bando
    UNION

    SELECT 
        conflictos.conflicto,
        bandos.bando,
        MAX(bandos.fecha_salida) as fecha_bando2
    FROM Conflictos
    LEFT JOIN bandos ON
     bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando2')
    WHERE fecha_salida > 0
    GROUP BY conflictos.conflicto,bandos.bando
    UNION

    SELECT 
        conflictos.conflicto,
        bandos.bando,
        MAX(bandos.fecha_salida) as fecha_bando3
    FROM Conflictos
    LEFT JOIN bandos ON
     bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando3')
    WHERE fecha_salida > 0
    GROUP BY conflictos.conflicto, bandos.bando
    UNION

    SELECT 
        conflictos.conflicto,
        bandos.bando,
        MAX(bandos.fecha_salida) as fecha_bando4
    FROM Conflictos
    LEFT JOIN bandos ON
     bandos.id_bando = JSON_EXTRACT(conflictos.id_bando_json_fk, '$.Bando4')
    WHERE fecha_salida > 0
    GROUP BY conflictos.conflicto, bandos.bando
    
    """)

# DATOS

In [ ]:
query="""INSERT INTO planetas (planeta) VALUES   ("Kamino"),  ("Geonosis"),  ("Utapau"),  ("Mustafar"),  ("Kashyyyk"),  ("Polis Massa"),  ("Mygeeto"),  ("Felucia"),  ("Cato Neimoidia"),  ("Christophsis"),  ("Aletta"),  ("Vallt"),  ("Florrum"),  ("Phindar"),  ("Kessel"),  ("Boz Pity"),  ("Anaxes"),  ("Galactic City"),  ("Lothal"),  ("Lotho Minor");
INSERT INTO causas (causa) VALUES  ("Territorios"), ("Religiones"), ("Suministros"), ("Razas");
INSERT INTO religiones (religion) VALUES ("La Fuerza"), ("Orden Jedi"), ("Orden Sith"), ("Templo de las Estrellas"), ("Círculo Cósmico"), ("Templo del Dragón Galáctico"), ("Dios del Universo"), ("Caminante del Cosmos"), ("Religión de los Eternos"), ("Orden de los Guerreros Galácticos");
INSERT INTO suministros (suministro) VALUES  ("Oro"), ("Plata"), ("Cristal Kyber"), ("Mineral Cortosis"), ("Gemas Poderosas"), ("Petroleo"), ("Gas"), ("Acero"), ("Titánio"), ("Níquel"), ("Alimentos"), ("Agua Potable");
INSERT INTO razas (raza) VALUES  ("Humanos"), ("Wookiees"), ("Twi'leks"), ("Ewoks"), ("Jawas"), ("Rodianos"), ("Gamorreanos"), ("Trandoshanos"), ("Hutts"), ("Kel Dor"), ("Zabrak"), ("Sullustanos"), ("Mon Calamari"), ("Togrutas"), ("Rattatakis"), ("Nautolanas"), ("Chiss"), ("Gran"), ("Gungan"), ("Duros");
INSERT INTO territorios (territorio) values  ("Mos Eisley"), ("Mos Espa"), ("Theed"), ("Mos Taike"), ("Mos Shuuta"), ("Mos Eisley Spaceport"), ("Mos Entha"), ("Cloud City"), ("Mos Eisley Cantina"), ("Coruscant"), ("Kaadara"), ("Theed Hangar"), ("Keren"), ("Chalmun's Spaceport Cantina"), ("Anchorhead"), ("Theed Royal Palace");
INSERT INTO lideres (lider) VALUES   ("Emperor Palpatine"), ("Darth Vader"), ("Grand Moff Tarkin"), ("Admiral Ackbar"), ("Mon Mothma"), ("Yoda"), ("Mace Windu"), ("Bail Organa"), ("Princess Leia"), ("Lando Calrissian"), ("Padmé Amidala"), ("Shaak Ti"), ("Aayla Secura"), ("Plo Koon"), ("Ki-Adi-Mundi"), ("Agen Kolar"), ("Saesee Tiin"), ("Kit Fisto"), ("Qui-Gon Jinn"), ("Count Dooku");
INSERT INTO tipo_armas (id_tipo_arma, tipo_arma) VALUES (1, 'Armas de cuerpo a cuerpo'),(2, 'Armas de disparo'),(3, 'Armas de destrucción masiva');

INSERT INTO armas (id_arma, arma, indicador_dest, id_tipo_arma_fk, id_comerciante_json_fk)VALUES 
      (1, 'Sable de luz', 9, 1, '{"Comerciante1":1,"Comerciante2":4}'),
      (2, 'Lanza de plasma', 7, 1, '{"Comerciante1":2,"Comerciante2":5,"Comerciante3":6}'),
      (3, 'Pistola blaster', 6, 2, '{"Comerciante1":7,"Comerciante2":9}'),
      (4, 'Arma de rayo', 8, 2, '{"Comerciante1":8,"Comerciante2":10}'),
      (5, 'Lanzagranadas', 8, 2, '{"Comerciante1":11}'),
      (6, 'Disintegrador', 10, 2, '{"Comerciante1":1,"Comerciante2":2,"Comerciante3":3,"Comerciante4":4,"Comerciante5":7}'),
      (7, 'Lanzacohetes', 9, 2, '{"Comerciante1":6,"Comerciante2":8,"Comerciante3":9}'),
      (8, 'Cañón de iones', 10, 2, '{"Comerciante1":10,"Comerciante2":11}'),
      (9, 'Flechas de plasma', 7, 2, '{"Comerciante1":1,"Comerciante2":2,"Comerciante3":3,"Comerciante4":4}'),
      (10, 'Cargador de explosivos', 9, 2, '{"Comerciante1":7,"Comerciante2":6,"Comerciante3":8}'),
      (11, 'Fusil de asalto blaster', 7, 2, '{"Comerciante1":9,"Comerciante2":10}'),
      (12, 'Cargador de láser', 8, 2, '{"Comerciante1":11}'),
      (13, 'Lanza de choque', 6, 2, '{"Comerciante1":1,"Comerciante2":2,"Comerciante3":3}'),
      (14, 'Dispositivo de destrucción planetaria', 10, 3, '{"Comerciante1":4,"Comerciante2":7,"Comerciante3":6,"Comerciante4":8}'),
      (15, 'Cetro láser', 8, 3, '{"Comerciante1":9,"Comerciante2":10,"Comerciante3":11}'),
      (16, 'Lanzamisiles', 9, 2, '{"Comerciante1":1,"Comerciante2":2,"Comerciante3":3}'),
      (17, 'Escopeta de blaster', 6, 2, '{"Comerciante1":4,"Comerciante2":7}'),
      (18, 'Arma de rayo láser', 9, 2, '{"Comerciante1":6,"Comerciante2":8,"Comerciante3":9}'),
      (19, 'Lanza de energía', 8, 1, '{"Comerciante1":10,"Comerciante2":11}'),
      (20, 'Dispositivo de minado', 7, 3, '{"Comerciante1":1,"Comerciante2":2,"Comerciante3":3,"Comerciante4":4}');

INSERT INTO comerciantes (id_comerciante, comerciante)
VALUES (1, 'Tobb Jadak'),
       (2, 'Figrin D''an'),
       (3, 'Wuher'),
       (4, 'Gha Nachkt'),
       (5, 'Watto'),
       (6, 'Teemo el Tasa'),
       (7, 'Jabba el Hutt'),
       (8, 'URoRRuR''R''R'),
       (9, 'Gorga Desilijic Aarrpo'),
       (10, 'Arsin Crassus');

INSERT INTO divisiones (id_division, division, naves, droides, veh_terrestres, soldados, bajas)
VALUES (1, 'Flota de la Fuerza', 10, 50, 5, 7000, 1000),
       (2, 'Flota del Caos', 20, 60, 10, 5000, 5000),
       (3, 'Flota de la Rebelión', 15, 70, 3, 4000, 2000),
       (4, 'Flota del Imperio', 25, 80, 8, 8000, 7000),
       (5, 'Flota de la Luz', 12, 55, 6, 3000, 1200),
       (6, 'Flota de las Tinieblas', 22, 65, 11, 9000, 5500),
       (7, 'Flota de la Esperanza', 17, 75, 4, 3500, 2200),
       (8, 'Flota de la Desesperación', 27, 85, 9, 8500, 7500),
       (9, 'Flota de la Paz', 14, 50, 7, 3200, 1400),
       (10, 'Flota de la Guerra', 24, 60, 12, 6000, 6000),
       (11, 'Flota de la Armonía', 19, 70, 5, 4000, 2400),
       (12, 'Flota del Desorden', 29, 80, 10, 9500, 8000),
       (13, 'Flota de la Justicia', 16, 55, 8, 3500, 1600),
       (14, 'Flota de la Injusticia', 26, 65, 13, 7000, 6500),
       (15, 'Flota de la Orden', 21, 75, 6, 4500, 2600),
       (16, 'Flota del Caos', 31, 85, 11, 9000, 8500),
       (17, 'Flota de la Unidad', 18, 50, 9, 6740, 1800),
       (18, 'Flota de la Diversidad', 28, 60, 14, 5280, 7000),
       (19, 'Flota de la Solidaridad', 23, 70, 7, 3190, 2800),
       (20, 'Flota del Egoísmo', 33, 80, 12, 8330, 9000);

INSERT INTO bandos (id_bando, bando, fecha_entrada, fecha_salida, id_division_json_fk, id_comerciante_json_fk, id_lider_json_fk)
VALUES (1, 'Imperio Galactico', '8000-11-25', '9320-12-31', '{"Division1":8,"Division2":3,"Division3":17}', '{"Comerciante1":1,"Comerciante2":3,"Comerciante3":4}', '{"Lider1":3,"Lider2":8}'),
       (2, 'Republica Galactica', '8000-11-26', '9320-12-31', '{"Division1":7,"Division2":20}', '{"Comerciante1":2,"Comerciante2":5,"Comerciante3":8}', '{"Lider1":6,"Lider2":19,"Lider3":17}'),
       (3, 'Alianza Rebelde', '8030-05-11', '9000-07-06', '{"Division1":12,"Division2":16,"Division3":3}', '{"Comerciante1":6,"Comerciante2":9}', '{"Lider1":20}'),
       (4, 'Mandalorianos', '9001-06-09', '9050-10-03', '{"Division1":5,"Division2":7,"Division3":8,"Division4":14}', '{"Comerciante1":7,"Comerciante2":10}', '{"Lider1":12,"Lider2":18,"Lider3":5}'),
       (5, 'Jedi', '8000-11-26', '9320-12-31', '{"Division1":17,"Division2":10,"Division3":15}', '{"Comerciante1":3,"Comerciante2":6}', '{"Lider1":10,"Lider2":16,"Lider3":4}'),
       (6, 'Sith', '8000-11-25', '9320-12-31', '{"Division1":13,"Division2":5}', '{"Comerciante1":1,"Comerciante2":5,"Comerciante3":10}', '{"Lider1":2}'),
       (7, 'Fuerzas de Autodefensa de la Nueva República', '8159-08-15', '8500-07-07', '{"Division1":15,"Division2":19,"Division3":14,"Division4":8}', '{"Comerciante1":4,"Comerciante2":7,"Comerciante3":8}', '{"Lider1":15,"Lider2":7}'),
       (8, 'Asociación de Comercio', '8023-11-11', '9000-05-03', '{"Division1":18,"Division2":20,"Division3":4}', '{"Comerciante1":2,"Comerciante2":9}', '{"Lider1":11,"Lider3":13}'),
       (9, 'Orden Jedi de la Antigua República', null, '9156-08-30', '{"Division1":9,"Division2":6,"Division3":20}', '{"Comerciante1":1,"Comerciante2":3}', '{"Lider1":1}'),
       (10, 'Orden Sith de la Antigua República', '8800-05-15', '9300-04-25', '{"Division1":1,"Division2":12,"Division3":6,"Division4":15}', '{"Comerciante1":5,"Comerciante2":10}', '{"Lider1":6,"Lider2":16,"Lider3":7}'),
       (11, 'Orden Jedi de los Jedi', '8000-11-26', '9320-12-31', '{"Division1":7,"Division2":2,"Division3":14}', '{"Comerciante1":6,"Comerciante2":8}', '{"Lider1":9,"Lider2":13,"Lider3":5}'),
       (12, 'Orden Sith de los Sith', '8030-05-11', '9320-12-31', '{"Division1":16,"Division2":9}', '{"Comerciante1":4,"Comerciante2":7}', '{"Lider1":17,"Lider2":8}'),
       (13, 'Cuerpo de Seguridad de la República', null, '9320-12-31', '{"Division1":4,"Division2":1,"Division3":8,"Division4":18}', '{"Comerciante1":2,"Comerciante2":5,"Comerciante3":9}', '{"Lider1":3,"Lider2":7,"Lider3":13}'),
       (14, 'Cazarrecompensas', '8000-11-26', '9320-12-31', '{"Division1":4,"Division2":5,"Division3":11}', '{"Comerciante1":2,"Comerciante2":6,"Comerciante3":9}', '{"Lider1":1,"Lider2":2,"Lider3":3}'),
       (15, 'Bando de Piratas', '8000-11-25', '9320-12-31', '{"Division1":8,"Division2":10,"Division3":13}', '{"Comerciante1":1,"Comerciante2":3,"Comerciante3":5}', '{"Lider1":4,"Lider2":6,"Lider3":8}'),
       (16, 'Bando de Piratas', '8300-11-25', '9220-11-11', '{"Division2":8,"Division2":10}', '{"Comerciante4":1,"Comerciante2":3,"Comerciante3":2}', '{"Lider1":2,"Lider2":6,"Lider3":1}');

INSERT INTO causa_conflictos (id_causa_conflicto, id_causa_fk, interes_json_fk)
VALUES (1, 1, '{"Territorio1":1,"Territorio2":7}'),
       (2, 1, '{"Territorio1":2,"Territorio2":8,"Territorio3":10,"Territorio4":7}'),
       (3, 2, '{"Religion1":1,"Religion2":2}'),
       (4, 2, '{"Religion1":4,"Religion2":6,"Religion3":9}'),
       (5, 3, '{"Suministro1":3,"Suministro2":2,"Suministro3":6,"Suministro4":4}'),
       (6, 3, '{"Suministro1":9,"Suministro2":8,"Suministro3":5,"Suministro4":3,"Suministro5":2,"Suministro6":7}'),
       (7, 4, '{"Raza1":1,"Raza2":2,"Raza3":6,"Raza4":11}'),
       (8, 4, '{"Raza1":1,"Raza2":12,"Raza3":4,"Raza4":1,"Raza5":11,"Raza6":5}');
       
INSERT INTO org_mediadoras (id_org, mediador, tipo, organizacion, fecha_entrada, fecha_salida, id_lider_json_fk)
VALUES (1, 'Ahsoka Tano', 'Jedi', 'Orden Jedi', '8045-06-12', '8136-12-22', '{"Lider1":1}'),
       (2, 'Kanan Jarrus', 'Jedi', 'Orden Jedi (en el exilio)', '8102-03-05', '8105-07-25', '{"Lider1":2}'),
       (3, 'Jango Fett', 'Cazarrecompensas', 'Independiente', '8198-11-18', '8203-06-07', '{"Lider1":3,"Lider2":11}'),
       (4, 'Luke Skywalker', 'Jedi', 'Orden Jedi', '8234-02-09', null, '{"Lider1":4}'),
       (5, 'Anakin Skywalker', 'Jedi', 'Orden Jedi', '8237-01-09', '8241-12-30', '{"Lider1":5}'),
       (6, 'Asajj Ventress', 'Sith', 'Orden Sith', '8280-05-21', '8247-09-30', '{"Lider1":6,"Lider2":10}'),
       (7, 'General Grievous', 'Cyborg', 'Droid Armada', '8290-11-15', null, '{"Lider1":7,"Lider2":16}'),
       (8, 'Supreme Leader Snoke', 'Sith', 'Primer Orden', '8362-02-28', '8235-07-05', '{"Lider1":8}'),
       (9, 'Rey', 'Jedi', 'Orden Jedi', '8488-06-15', '8599-12-25', '{"Lider1":9,"Lider2":1}'),
       (10, 'Finn', 'Stormtrooper desertor', 'Fuerzas de la Alianza Rebelde', '8494-03-08', '8503-12-19', '{"Lider1":10,"Lider2":7}'),
       (11,'Poe Dameron','Piloto','Fuerzas de la Alianza Rebelde','8491-06-20','8502-02-23','{"Lider1":11}'),
       (12,'Rose Tico','Ingeniera','Fuerzas de la Alianza Rebelde','8490-11-20','8501-06-10','{"Lider1":12,"Lider2":8}'),
       (13,'Maz Kanata','Líder pirate','Independiente','8557-09-05',null,'{"Lider1":13}'),
       (14,'BB-8','Droide','Fuerzas de la Alianza Rebelde','8537-01-13','8542-07-18','{"Lider1":14}'),
       (15,'R2-D2','Droide','Orden Jedi','8584-05-26','8588-07-05','{"Lider1":15}');
       
INSERT INTO conflictos (id_conflicto, Conflicto, muertos, heridos, id_causa_conflicto_fk, id_planeta_fk, id_org_fk, id_bando_json_fk)
VALUES (1, 'Batalla de Geonosis', 3940, 14560, 1, 11, 12, '{"Bando1":1,"Bando2":2}'),
       (2, 'Batalla de Coruscant', 108, 8168, 2, 9, 5, '{"Bando1":3,"Bando2":4,"Bando3":5}'),
       (3, "Batalla de Hoth",782, 24335, 3, 4,null, '{"Bando1":6,"Bando2":7,"Bando3":8}'),
       (4, "Batalla de Endor",384, 7713, 4, 7, 5, '{"Bando1":9,"Bando2":10,"Bando3":11}'),
       (5, "Batalla de Christophsis",376, 27948, 5, 19, 13, '{"Bando1":12,"Bando2":13,"Bando3":14}'),
       (6, "Batalla de Felucia",375, 16564, 6, 17, 5, '{"Bando1":15,"Bando2":1,"Bando3":2}'),
       (7, "Batalla de Naboo",103, 17360, 7, 8, null, '{"Bando1":16,"Bando2":4,"Bando3":5}'),
       (8, 'Batalla de Kashyyyk', 565, 29342, 8, 14, null, '{"Bando1":6,"Bando2":7,"Bando3":8}'),
       (9, 'Batalla de Mustafar', 578, 19015, 1, 13, 5, '{"Bando1":9,"Bando2":10,"Bando3":11}'),
       (10, 'Batalla de Mon Cala', 578, 6252, 2, 20, null, '{"Bando1":12,"Bando2":13,"Bando3":14}'),
       (11, 'Batalla de Yavin', 775, 6152, 3, 3, 5, '{"Bando1":15,"Bando2":1,"Bando3":2}'),
       (12, 'Batalla de Dantooine', 148, 2389, 4, 6, 12, '{"Bando1":3,"Bando2":4,"Bando3":5}'),
       (13, 'Batalla de Dathomir', 359, 14264, 5, 3, 3, '{"Bando1":6,"Bando2":7,"Bando3":8}'),
       (14, 'Batalla de Scarif', 299, 12459, 6, 4, null, '{"Bando1":9,"Bando2":10,"Bando3":11}'),
       (15, 'Batalla de Hissrich', 161, 1429, 7, 7, 4, '{"Bando1":12,"Bando2":13,"Bando3":14}'),
       (16, 'Batalla de Korsinata', 295, 2943, 8, 18, null, '{"Bando1":5,"Bando2":6,"Bando3":7,"Bando4":8}'),
       (17, 'Batalla de Ryloth', 463, 23486, 1, 5, 1, '{"Bando1":9,"Bando2":10,"Bando3":11,"Bando4":12}'),
       (18, 'Batalla de Kamino', 818, 6758, 2, 10, null, '{"Bando1":13,"Bando2":14,"Bando3":15,"Bando4":1}'),
       (19, 'Batalla de Utapau', 747, 3941, 3, 12, 10, '{"Bando1":2,"Bando2":3,"Bando3":4,"Bando4":5}'),
       (20, 'Batalla de Saleucami', 309, 19575, 4, 15, 1, '{"Bando1":6,"Bando2":7,"Bando3":8,"Bando4":9}');
"""


